In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
print(os.listdir("../input"))
np.random.seed(5)
# Any results you write to the current directory are saved as output.

In [ ]:
x1c1=np.random.random((1000,1))
x2c1=-5*x1c1+3+(np.random.randint(3,size=(1000,1))+1)*np.random.random((1000,1))
y1=np.ones((1000,1))
x1c2=np.random.random((1000,1))
x2c2=-5*x1c2+3-(np.random.randint(3,size=(1000,1))+1)*np.random.random((1000,1))
y2=np.zeros((1000,1))
df1=np.concatenate((x1c1,x2c1),axis=1)
df2=np.concatenate((x1c2,x2c2),axis=1)
df1=np.concatenate((df1,df2),axis=0)
y1=np.concatenate((y1,y2),axis=0)
df1=np.concatenate((df1,y1),axis=1)
np.random.shuffle(df1)
y1=df1[:,2].flatten().astype(int)
print(y1)
y1=np.eye(2)[y1]
df1=df1[:,:2]
print(df1)
trainX=df1
trainY=y1
trainX,testX,trainY,testY=train_test_split(trainX,trainY,test_size=0.3,stratify=trainY,random_state=2)
testX,valX,testY,valY=train_test_split(testX,testY,test_size=0.33,stratify=testY,random_state=2)

In [ ]:
'''trainX=pd.read_csv("../input/iris.data.csv",header=None,names=["x1","x2","x3","x4","y"])
trainY=trainX["y"]
trainY=LabelEncoder().fit_transform(trainY.values)
trainY=np.eye(3)[trainY]
trainX=trainX.drop(["y"],axis=1).values
trainX=MinMaxScaler().fit_transform(trainX)
trainX,testX,trainY,testY=train_test_split(trainX,trainY,test_size=0.3,stratify=trainY,random_state=2)
testX,valX,testY,valY=train_test_split(testX,testY,test_size=0.33,stratify=testY,random_state=2)'''


In [ ]:
print(trainX.shape,trainY.shape)
print(valX.shape,valY.shape)
print(testX.shape,testY.shape)

In [ ]:
label=np.argmax(trainY,axis=1)
colors=["orange","black"]
plt.scatter(trainX[:,0], trainX[:,1], c=label, cmap=matplotlib.colors.ListedColormap(colors))


In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

class Model:
    chkptw=None
    def __init__(self):
        self.layer_count=0
        self.weights={}
        self.biases={}
        self.l_dims=[]
        self.grad_W={}
        self.grad_b={}
        self.layer_ops={}
        self.summary={}
        self.history={"train_loss":[],"val_loss":[],"train_acc":[],"val_acc":[]}
        
    def Dense(self,nodes=1,input_shape=None):
        self.layer_count+=1
        if input_shape:
            self.l_dims.append(input_shape)
        self.l_dims.append(nodes)
        return np.random.random_sample((nodes,self.l_dims[self.layer_count-1])),np.random.random_sample((nodes,1))
        
    def add(self,weights):
        self.weights[f"w_{self.layer_count}"]=weights[0]
        self.biases[f"b_{self.layer_count}"]=weights[1]
        self.summary[f"Dense_{self.layer_count}"]=weights[0].shape
        
    def predict(self,x):
        self.layer_ops["v_0"]=x
        for i in range(self.layer_count):
            x=self.layer_ops[f"v_{i}"]
            x=np.dot(x,self.weights[f'w_{i+1}'].T)
            x=sigmoid(x)
            self.layer_ops[f'v_{i+1}']=np.add(x,self.biases[f"b_{i+1}"].T)
        return x
    
    def fit(self,X,Y,validation_data=None,epochs=100000,batch_size=10,lr=0.005):
        batches=X.shape[0]//batch_size
        best_acc=0
        if validation_data:
            valX,valY=validation_data
        for e in range(epochs):
            loss=0
            for i in range(batches):
                x=X[i*batch_size:i+1*batch_size]
                y=Y[i*batch_size:i+1*batch_size]
                v=self.predict(x)
                grad_base=((v-y)*(1-v)*v).T
                for j in reversed(range(1,self.layer_count+1)):
                    self.grad_W[f'{"w_"}{j}']=grad_base
                    for k in reversed(range(j,self.layer_count)):
                        self.grad_W[f'{"w_"}{j}']=np.dot(self.weights[f'{"w_"}{k+1}'].T,self.grad_W[f'{"w_"}{j}'])
                        self.grad_W[f'{"w_"}{j}']=self.grad_W[f'{"w_"}{j}']*(self.layer_ops[f'{"v_"}{k}']*(1-self.layer_ops[f'{"v_"}{k}'])).T
                    self.grad_b[f"b_{j}"]=np.sum(self.grad_W[f"w_{j}"],axis=1).reshape(self.biases[f"b_{j}"].shape)
                    self.grad_W[f'{"w_"}{j}']=np.dot(self.grad_W[f'{"w_"}{j}'],self.layer_ops[f'{"v_"}{j-1}'])
                    self.weights[f'{"w_"}{j}']-=lr*self.grad_W[f'{"w_"}{j}']
                    self.biases[f'b_{j}']-=lr*self.grad_b[f"b_{j}"]
                loss+=np.sum((v-y)*(v-y))/(2*batch_size)
            val_loss=0
            train_acc=accuracy_score(np.argmax(trainY,axis=1),np.argmax(self.predict(trainX),axis=1))
            self.history["train_acc"].append(train_acc)
            self.history["train_loss"].append(loss)
            if validation_data:
                val_v=self.predict(valX)
                val_loss+=np.sum((val_v-valY)*(val_v-valY))/(2*valX.shape[0])
                val_acc=accuracy_score(np.argmax(valY,axis=1),np.argmax(self.predict(valX),axis=1))
                self.history["val_acc"].append(val_acc)
                self.history["val_loss"].append(val_loss)
                if best_acc<val_acc:
                    best_acc=val_acc
                    chkptw=self
                print(f"epoch {e}  train loss: {loss}  validation loss:{val_loss}")
            else:
                print(f"epoch {e}  train loss: {loss} ")
        return chkptw

In [ ]:
model=Model()
model.add(model.Dense(nodes=4,input_shape=trainX.shape[1]))
model.add(model.Dense(nodes=3))
model.add(model.Dense(trainY.shape[1]))
model.fit(trainX,trainY,validation_data=[valX,valY],epochs=6000,batch_size=50)

In [ ]:
plt.plot(model.history['train_acc'])
plt.plot(model.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(model.history['train_loss'])
plt.plot(model.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
accuracy_score(np.argmax(testY,axis=1),np.argmax(model.predict(testX),axis=1))

In [ ]:
for key,values in model.biases.items():
    print(key,values.shape)

In [ ]:
for key,values in model.layer_ops.items():
    print(key,values.shape)

In [ ]:
for key,values in model.weights.items():
    print(key,values.shape)

In [ ]:
model.biases

In [ ]:
chkptw.biases

In [ ]:
trainX.shape

In [ ]:
trainY.shape

In [ ]:
np.argmax(chkptw.predict(testX),axis=1)